In [1]:
from pyspark.sql import SparkSession

catalog_name = "rainbow-data-production-iceberg-rest-cv"
project_id = "rainbow-data-production-483609"
bucket_name = "rainbow-data-production-iceberg-rest-cv"
spark = (
    SparkSession.builder.appName("Spark-Iceberg")
    .config("spark.master", "local[*]")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.driver.port", "7078")
    .config("spark.blockManager.port", "7079")
    .config("spark.driver.memory", "2G")
    .config("spark.executor.memory", "2G")
    .config("spark.driver.userClassPathFirst", "false")
    .config("spark.executor.userClassPathFirst", "false")
    .config(
        f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog"
    )
    .config(f"spark.sql.catalog.{catalog_name}.type", "rest")
    .config(
        f"spark.sql.catalog.{catalog_name}.uri",
        "https://biglake.googleapis.com/iceberg/v1/restcatalog",
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.warehouse",
        f"gs://{bucket_name}",
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.header.x-goog-user-project",
        project_id,
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.rest.auth.type",
        "org.apache.iceberg.gcp.auth.GoogleAuthManager",
    )
    .config(
        f"spark.sql.catalog.{catalog_name}.io-impl",
        "org.apache.iceberg.gcp.gcs.GCSFileIO",
    )
    .config(f"spark.sql.catalog.{catalog_name}.rest-metrics-reporting-enabled", "false")
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    )
    .config("spark.sql.defaultCatalog", catalog_name)
    .config(f'spark.sql.catalog.{catalog_name}.header.X-Iceberg-Access-Delegation','vended-credentials')
    .config(
        "spark.jars.packages",
        ",".join(
            [
                "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.10.1",
                "org.apache.iceberg:iceberg-gcp-bundle:1.10.1",
                # "com.google.auth:google-auth-library-oauth2-http:1.41.0",
                # "com.google.auth:google-auth-library-credentials:1.41.0",
                # "com.google.guava:guava:32.1.2-jre",
                # "com.google.cloud:google-cloud-storage:2.61.0",
                # "com.google.cloud:libraries-bom:26.73.0",
            ]
        ),
    )
    .getOrCreate()
)


:: loading settings :: url = jar:file:/Users/tuan.tran1/Workspaces/Test/TestIceberg/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tuan.tran1/.ivy2/cache
The jars for the packages stored in: /Users/tuan.tran1/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-gcp-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b4e05d8b-e7fa-473d-aa2b-aba6c5376408;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.10.1 in central
	found org.apache.iceberg#iceberg-gcp-bundle;1.10.1 in central
:: resolution report :: resolve 67ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-gcp-bundle;1.10.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.10.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------------------

In [2]:
spark.sql("SHOW CATALOGS").show()

+--------------------+
|             catalog|
+--------------------+
|rainbow-data-prod...|
|       spark_catalog|
+--------------------+



In [3]:
spark.sql("SELECT current_catalog();").show()

+--------------------+
|   current_catalog()|
+--------------------+
|rainbow-data-prod...|
+--------------------+



In [4]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS test_namespace_rest_cv ;")
# spark.sql("CREATE NAMESPACE IF NOT EXISTS test_namespace_rest_cv LOCATION 'gs://rainbow-data-production-iceberg/test_namespace_rest_cv' WITH DBPROPERTIES ('gcp-region' = 'us-central1');")

DataFrame[]

In [5]:
spark.sql("SHOW NAMESPACES").show()

+--------------------+
|           namespace|
+--------------------+
|test_namespace_re...|
+--------------------+



In [4]:
spark.sql("USE test_namespace_rest_cv;")

DataFrame[]

In [9]:
spark.sql("CREATE TABLE IF NOT EXISTS sample_table (id BIGINT, data STRING) USING ICEBERG;")

DataFrame[]

In [5]:
spark.sql("SHOW TABLES").show()

+--------------------+------------+-----------+
|           namespace|   tableName|isTemporary|
+--------------------+------------+-----------+
|test_namespace_re...|sample_table|      false|
+--------------------+------------+-----------+



In [11]:
spark.sql("""
INSERT INTO sample_table VALUES
  (1, 'first'), (2, 'second'), (3, 'third')
""")

DataFrame[]

In [6]:
spark.sql("SELECT * FROM sample_table").show()

+---+------+
| id|  data|
+---+------+
|  1| first|
|  2|second|
|  3| third|
|  4|fourth|
|  5| fifth|
|  6| sixth|
+---+------+



26/01/08 20:43:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1061703 ms exceeds timeout 120000 ms
26/01/08 20:43:38 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/08 21:00:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [13]:
spark.sql("""
INSERT INTO sample_table VALUES
  (4, 'fourth'), (5, 'fifth'), (6, 'sixth');
""")

DataFrame[]

In [15]:
spark.sql("SELECT * FROM sample_table;").show()

+---+------+
| id|  data|
+---+------+
|  1| first|
|  2|second|
|  3| third|
|  4|fourth|
|  5| fifth|
|  6| sixth|
+---+------+

